In [1]:
import torch

def create_sparse_pauli():
    """
    Create sparse representations of Pauli matrices.
    Returns sparse versions of σx, σy, σz, and Identity.
    """
    # For σx (off-diagonal elements)
    i_x = torch.tensor([[0, 1], [1, 0]])  # Row indices
    j_x = torch.tensor([[1, 0], [0, 1]])  # Column indices
    v_x = torch.tensor([1., 1.])  # Values
    sigma_x = torch.sparse_coo_tensor(
        indices=torch.stack([i_x.flatten(), j_x.flatten()]),
        values=v_x,
        size=(2, 2)
    )

    # For σy (off-diagonal elements with imaginary parts)
    i_y = torch.tensor([[0, 1], [1, 0]])
    j_y = torch.tensor([[1, 0], [0, 1]])
    v_y = torch.tensor([[-1j, 1j]])  # Complex values
    sigma_y = torch.sparse_coo_tensor(
        indices=torch.stack([i_y.flatten(), j_y.flatten()]),
        values=v_y,
        size=(2, 2)
    )

    # For σz (diagonal elements)
    i_z = torch.tensor([[0, 1]])
    j_z = torch.tensor([[0, 1]])
    v_z = torch.tensor([1., -1.])
    sigma_z = torch.sparse_coo_tensor(
        indices=torch.stack([i_z.flatten(), j_z.flatten()]),
        values=v_z,
        size=(2, 2)
    )

    # Identity matrix
    i_I = torch.tensor([[0, 1]])
    j_I = torch.tensor([[0, 1]])
    v_I = torch.tensor([1., 1.])
    I = torch.sparse_coo_tensor(
        indices=torch.stack([i_I.flatten(), j_I.flatten()]),
        values=v_I,
        size=(2, 2)
    )

    return sigma_x, sigma_y, sigma_z, I

def sparse_kron(A, B):
    """
    Compute Kronecker product of two sparse matrices efficiently.
    
    Args:
        A: First sparse tensor
        B: Second sparse tensor
    
    Returns:
        Sparse tensor representing the Kronecker product
    """
    # Get indices and values of non-zero elements
    A_indices = A._indices()
    B_indices = B._indices()
    A_values = A._values()
    B_values = B._values()
    
    # Calculate sizes
    A_size = torch.tensor(A.size())
    B_size = torch.tensor(B.size())
    
    # Create new indices for Kronecker product
    nnz_A = A_values.size(0)
    nnz_B = B_values.size(0)
    
    # Calculate new indices using broadcasting
    kron_row_A = A_indices[0].repeat_interleave(nnz_B) * B_size[0]
    kron_row_B = B_indices[0].repeat(nnz_A)
    kron_col_A = A_indices[1].repeat_interleave(nnz_B) * B_size[1]
    kron_col_B = B_indices[1].repeat(nnz_A)
    
    # Combine indices
    kron_indices = torch.stack([
        kron_row_A + kron_row_B,
        kron_col_A + kron_col_B
    ])
    
    # Calculate new values
    kron_values = torch.outer(A_values, B_values).flatten()
    
    # Create new sparse tensor
    return torch.sparse_coo_tensor(
        indices=kron_indices,
        values=kron_values,
        size=(A_size[0] * B_size[0], A_size[1] * B_size[1])
    )

def get_sparse_pauli_operator(operator_type, position, n_spins):
    """
    Create a sparse operator for a Pauli matrix acting on a specific spin.
    
    Args:
        operator_type: 'x', 'y', or 'z' for the type of Pauli matrix
        position: Which spin the operator acts on
        n_spins: Total number of spins in the system
    
    Returns:
        Sparse tensor representing the operator on the full system
    """
    # Get sparse Pauli matrices
    sigma_x, sigma_y, sigma_z, I = create_sparse_pauli()
    
    # Select appropriate operator
    if operator_type == 'x':
        op = sigma_x
    elif operator_type == 'y':
        op = sigma_y
    elif operator_type == 'z':
        op = sigma_z
    else:
        raise ValueError(f"Unknown operator type: {operator_type}")
    
    # Start with identity
    result = torch.sparse_coo_tensor(
        indices=torch.tensor([[0], [0]]),
        values=torch.tensor([1.]),
        size=(1, 1)
    )
    
    # Build operator using Kronecker products
    for i in range(n_spins):
        if i == position:
            result = sparse_kron(result, op)
        else:
            result = sparse_kron(result, I)
    
    return result

# Example usage and demonstration of memory savings
def compare_dense_vs_sparse(n_spins=6):
    """Compare memory usage of dense vs sparse representations."""
    # Create operators
    sparse_op = get_sparse_pauli_operator('x', 0, n_spins)
    dense_op = sparse_op.to_dense()
    
    # Calculate memory usage
    sparse_memory = (sparse_op._values().nelement() * sparse_op._values().element_size() +
                    sparse_op._indices().nelement() * sparse_op._indices().element_size())
    dense_memory = dense_op.nelement() * dense_op.element_size()
    
    print(f"System size: {n_spins} spins")
    print(f"Dense matrix size: {dense_op.size()}")
    print(f"Number of non-zero elements: {sparse_op._values().nelement()}")
    print(f"Memory usage:")
    print(f"  Dense:  {dense_memory/1024:.2f} KB")
    print(f"  Sparse: {sparse_memory/1024:.2f} KB")
    print(f"Memory reduction factor: {dense_memory/sparse_memory:.1f}x")

# Run comparison
compare_dense_vs_sparse(6)

RuntimeError: indices and values must have same nnz, but got nnz from indices: 4, nnz from values: 2